In [1]:
import os
import numpy as np
import csv

In [2]:
# Set the Neper path
neper_path = "/home/jbarsima/software/neper/bin"
os.environ["PATH"] += os.pathsep + neper_path

# Need to module load gcc gsl

In [3]:
# Set the random seed for reproducibility
np.random.seed(27)

# Define the limits for each variable
D_limits = (0.02, 0.14)
Dstd_limits = (0.001, 0.02)
S_limits = (0.05, 0.25)
Sstd_limits = (0.001, 0.04)
l0 = 0.5
itermaxx = 30000

# Generate random samples using Latin hypercube sampling
n_samples = 7500  # change as desired
samples = np.zeros((n_samples, 4))
samples[:, 0] = np.linspace(D_limits[0], D_limits[1], n_samples) + np.random.uniform(0, 1/n_samples, n_samples)
samples[:, 1] = np.linspace(Dstd_limits[0], Dstd_limits[1], n_samples) + np.random.uniform(0, 1/n_samples, n_samples)
samples[:, 2] = np.linspace(S_limits[0], S_limits[1], n_samples) + np.random.uniform(0, 1/n_samples, n_samples)
samples[:, 3] = np.linspace(Sstd_limits[0], Sstd_limits[1], n_samples) + np.random.uniform(0, 1/n_samples, n_samples)

# Shuffle the samples to ensure randomness
np.random.shuffle(samples)

# Assign the variables to their respective samples
D = samples[:, 0]
Dstd = samples[:, 1]
S = samples[:, 2]
Sstd = samples[:, 3]

# Save the samples to a CSV file
with open('samples.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['D', 'Dstd', 'S', 'Sstd'])
    for i in range(n_samples):
        writer.writerow([D[i], Dstd[i], S[i], Sstd[i]])

# Call Neper for each sample
for i in range(n_samples):
    N = str(i+1)  # convert sample number to string
    cmd1 = f"neper -T -n from_morpho -morpho 'diameq:lognormal({D[i]},{Dstd[i]}),1-sphericity:lognormal({S[i]},{Sstd[i]})' -id 1 -periodicity all -domain 'cube({l0},{l0},{l0})' -morphooptistop itermax={itermaxx} -statcell vol,area -o RVE_{N}"
    os.system(cmd1)